In [1]:
import os, sys
import numpy as np
import xarray as xr
import pandas as pd

In [3]:
filename =  '/root/data/rrr/integrated_weather_dataset/data/raw/Guan_AR_Catalog/globalARcatalog_MERRA2_1980-2023_v4.0.nc'
ds = xr.open_dataset(filename, chunks={'time': 1460}, engine='netcdf4')
ds = ds.squeeze()
ds = ds.reset_coords(names=['lev', 'ens'], drop=True)

/tmp/ipykernel_2509/357022156.py:2: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1460. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(filename, chunks={'time': 1460}, engine='netcdf4')


In [4]:
# Step 2: Select the relevant geographical region
MIN_LON = -120+360
MAX_LON = -115+360
MIN_LAT = 31.5
MAX_LAT = 38
start_date = '2016-01-01T00:00:00.000000000'
end_date = '2016-12-31T00:00:00.000000000'
ds = ds.sel(time=slice(start_date, end_date), lat=slice(MIN_LAT,MAX_LAT), lon=slice(MIN_LON,MAX_LON))
print(f"Data selected for date : {start_date} to {end_date} lat: {MIN_LAT} to {MAX_LAT} and lon: {MIN_LON} to {MAX_LON}")


Data selected for date : 2016-01-01T00:00:00.000000000 to 2016-12-31T00:00:00.000000000 lat: 31.5 to 38 and lon: 240 to 245


In [5]:
ds

<xarray.Dataset> Size: 13MB
Dimensions:      (lon: 9, lat: 14, time: 1461)
Coordinates:
  * lon          (lon) float64 72B 240.0 240.6 241.2 241.9 ... 243.8 244.4 245.0
  * lat          (lat) float64 112B 31.5 32.0 32.5 33.0 ... 36.5 37.0 37.5 38.0
  * time         (time) datetime64[ns] 12kB 2016-01-01 ... 2016-12-31
Data variables: (12/55)
    islnd        (lat, lon) float32 504B dask.array<chunksize=(14, 9), meta=np.ndarray>
    iscst        (lat, lon) float32 504B dask.array<chunksize=(14, 9), meta=np.ndarray>
    ivt_begin    (lat, lon) float64 1kB dask.array<chunksize=(14, 9), meta=np.ndarray>
    ivt_end      (lat, lon) float64 1kB dask.array<chunksize=(14, 9), meta=np.ndarray>
    shapemap     (time, lat, lon) float32 736kB dask.array<chunksize=(1424, 14, 9), meta=np.ndarray>
    axismap      (time, lat, lon) float64 1MB dask.array<chunksize=(1424, 14, 9), meta=np.ndarray>
    ...           ...
    kinstveldir  (time, lat) float32 82kB dask.array<chunksize=(1424, 14), meta=np.ndarray>
    axisidx      (time, lat, lon) float32 736kB dask.array<chunksize=(1424, 14, 9), meta=np.ndarray>
    axislon      (time, lat, lon) float32 736kB dask.array<chunksize=(1424, 14, 9), meta=np.ndarray>
    axislat      (time, lat, lon) float32 736kB dask.array<chunksize=(1424, 14, 9), meta=np.ndarray>
    numobj       (time, lat, lon) float32 736kB dask.array<chunksize=(1424, 14, 9), meta=np.ndarray>
    intblosto    (lat, lon) float64 1kB dask.array<chunksize=(14, 9), meta=np.ndarray>
Attributes:
    title:                 Global Atmospheric River Database
    version:               2024.05.17
    creation_date:         17-May-2024 11:53:44
    tARget_version:        1.312
    dimension_convention:  5-d (nlon x nlat x nlev x ntime x nens) variables ...
    references:            (1) Guan, B., and D. E. Waliser (2015), Detection ...

In [13]:
df = ds.ivtx.to_dataframe(dim_order=['time', 'lat'])
df = df.reset_index()
df['time'] = pd.to_datetime(df['time'])
df

,time,lat,ivtx
0,2016-01-01,31.5,NaN
1,2016-01-01,32.0,NaN
2,2016-01-01,32.5,NaN
3,2016-01-01,33.0,NaN
4,2016-01-01,33.5,NaN
...,...,...,...
20449,2016-12-31,36.0,NaN
20450,2016-12-31,36.5,NaN
20451,2016-12-31,37.0,NaN
20452,2016-12-31,37.5,NaN


In [6]:
df['time'].unique()

<DatetimeArray>
['2016-01-03 18:00:00', '2016-01-04 00:00:00', '2016-01-04 06:00:00',
 '2016-01-04 12:00:00', '2016-01-04 18:00:00', '2016-01-05 06:00:00',
 '2016-01-05 12:00:00', '2016-01-05 18:00:00', '2016-01-06 00:00:00',
 '2016-01-06 06:00:00',
 ...
 '2016-12-22 18:00:00', '2016-12-23 00:00:00', '2016-12-23 12:00:00',
 '2016-12-23 18:00:00', '2016-12-24 00:00:00', '2016-12-24 06:00:00',
 '2016-12-24 12:00:00', '2016-12-24 18:00:00', '2016-12-30 12:00:00',
 '2016-12-31 00:00:00']
Length: 303, dtype: datetime64[ns]

In [4]:
df = ds.shapemap.to_dataframe(dim_order=['time', 'lat', 'lon']).dropna()
df = df.reset_index()
df['time'] = pd.to_datetime(df['time'])
df['Guan_AR_Label'] = df['shapemap'].notna().astype(int)
all_times = pd.date_range(start=start_date, end=end_date, freq='6h') 
all_combinations = pd.MultiIndex.from_product(
    [all_times, df['lat'].unique(), df['lon'].unique()],
    names=['time', 'lat', 'lon']
).to_frame(index=False)
print(f"Generated all combinations of time, latitude, and longitude: {len(all_combinations)} rows")


Generated all combinations of time, latitude, and longitude: 368046 rows


In [5]:
merged_df = pd.merge(
    all_combinations,
    df,
    on=['time', 'lat', 'lon'],
    how='left'
).fillna({'Guan_AR_Label': 0})
print("Data merged with all combinations.")
merged_df.rename(columns={'time': 'Timestamp', 'lat': 'Latitude', 'lon': 'Longitude'}, inplace=True)
merged_df['Longitude'] = ((merged_df['Longitude'] + 180) % 360) - 180
df_final = merged_df[['Timestamp', 'Latitude', 'Longitude', 'Guan_AR_Label']]


Data merged with all combinations.


In [6]:
df_final

,Timestamp,Latitude,Longitude,Guan_AR_Label
0,2016-01-01,31.5,-120.000,0.0
1,2016-01-01,31.5,-119.375,0.0
2,2016-01-01,31.5,-118.750,0.0
3,2016-01-01,31.5,-118.125,0.0
4,2016-01-01,31.5,-117.500,0.0
...,...,...,...,...
368041,2016-12-31,38.0,-117.500,0.0
368042,2016-12-31,38.0,-116.875,0.0
368043,2016-12-31,38.0,-116.250,0.0
368044,2016-12-31,38.0,-115.000,0.0


In [7]:
df2 = pd.read_csv('/gnn/rrr/ES3-TACLS/AR/dataset/labels/2016.csv', index_col = False)
df2['time'] = pd.to_datetime(df2['time'])
all_times_2 = pd.date_range(start=start_date, end=end_date, freq='3h')  # 3-hour intervals
all_combinations_2 = pd.MultiIndex.from_product(
    [all_times_2, df2['lat'].unique(), df2['lon'].unique()],
    names=['time', 'lat', 'lon']
).to_frame(index=False)

In [8]:
print(f"Generated all combinations of time, latitude, and longitude: {len(all_combinations_2)} rows")
all_combinations_2['Guan_AR_Label'] = 0
concat_df = pd.concat([all_combinations_2, df2], ignore_index=True)
result_df = concat_df.drop_duplicates(subset=['time', 'lat', 'lon'], keep='last')
result_df['Guan_AR_Label'] = result_df['Guan_AR_Label'].fillna(0)

Generated all combinations of time, latitude, and longitude: 368046 rows


/tmp/ipykernel_2892/67108147.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Guan_AR_Label'] = result_df['Guan_AR_Label'].fillna(0)


In [9]:
result_df

,time,lat,lon,Guan_AR_Label,Unnamed: 0,Rutz_AR_Label
0,2016-01-01,31.5,-120.000,0,NaN,NaN
1,2016-01-01,31.5,-119.375,0,NaN,NaN
2,2016-01-01,31.5,-118.750,0,NaN,NaN
3,2016-01-01,31.5,-118.125,0,NaN,NaN
4,2016-01-01,31.5,-117.500,0,NaN,NaN
...,...,...,...,...,...,...
402691,2016-12-31,34.0,-115.000,1,34645.0,0.0
402692,2016-12-31,34.5,-115.000,1,34646.0,0.0
402693,2016-12-31,35.0,-115.000,1,34647.0,0.0
402694,2016-12-31,35.5,-115.625,1,34648.0,0.0


In [10]:
df_final['Guan_AR_Label'].value_counts()


Guan_AR_Label
0.0    352563
1.0     15483
Name: count, dtype: int64

In [11]:
result_df['Guan_AR_Label'].value_counts()

Guan_AR_Label
0    335368
1     32678
Name: count, dtype: int64

In [12]:
df_final['Guan_AR_Label'] = result_df['Guan_AR_Label']


/tmp/ipykernel_2892/1729815312.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Guan_AR_Label'] = result_df['Guan_AR_Label']


In [13]:
df_final.reset_index(drop=True, inplace=True)
result_df.reset_index(drop=True, inplace=True)

df_final['Guan_AR_Label'] = result_df['Guan_AR_Label']


/tmp/ipykernel_2892/2442227332.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Guan_AR_Label'] = result_df['Guan_AR_Label']


In [14]:
result_df['Guan_AR_Label'].value_counts()

Guan_AR_Label
0    335368
1     32678
Name: count, dtype: int64

In [ ]:
result_df.to_csv('